In [ ]:
from pdf2image import convert_from_path
import os
import pytesseract
import fitz  # PyMuPDF
import requests
import pandas as pd
import time  # import time module for pausing
import json

# Set up tesseract
# pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'
# os.environ['TESSDATA_PREFIX'] = r'C:/Program Files/Tesseract-OCR/tessdata'  # Update this path to your Tesseract tessdata directory


In [ ]:
def extract_text_from_pdf(pdf_path, timeout=60):
    doc = fitz.open(pdf_path)
    text = ""
    start_time = time.time()
    for page in doc:
        if time.time() - start_time > timeout:
            print(f"Timeout reached for {pdf_path}")
            break
        text += page.get_text()
    return text


def extract_text_from_images(pdf_path, poppler_path=None): # , poppler_path=r'C:/Program Files/poppler/Release-24.02.0-0/poppler-24.02.0/Library\bin'
    if poppler_path:
        images = convert_from_path(pdf_path, poppler_path=poppler_path) 
    else:
        images = convert_from_path(pdf_path)
    
    text = ""
    for image in images:
        text += pytesseract.image_to_string(image, lang='deu')  # Use 'deu' for German texts
    return text


def categorize_invoice(pdf_path):
    keywords_electricity = ["Strom", "kWh", "Netzbetreiber", "Energie"]
    keywords_gas = ["Gas", "m³", "Heizkosten", "Gasanbieter", "therm", "brennwert", "kWh/m³", "Gasverbrauch", "Gasrechnung"]

    with fitz.open(pdf_path) as doc:
        text = ""
        for page in doc:
            text += page.get_text()

    text = text.lower()

    electricity_score = sum(keyword.lower() in text for keyword in keywords_electricity)
    gas_score = sum(keyword.lower() in text for keyword in keywords_gas)

    if electricity_score > gas_score:
        return "Strom"
    elif gas_score > electricity_score:
        return "Gas"
    else:
        return "Nicht eindeutig"


def process_pdf(file_path):
    text = extract_text_from_pdf(file_path)
    if not text.strip():
        text = extract_text_from_images(file_path)  # Use OCR if no text was found
    data = extract_data_from_text(text)
    if data is None:
        print(f"Failed to extract data from {file_path}")
    print(f"Extracted data from text for {file_path}:\n{data}\n")  # Debugging output
    return data


def extract_dict_from_response(response):
    if response is None:
        return None
    try:
        response = response.split('{',1)[1].split('}',1)[0]
        response = '{' + response + '}'
        return json.loads(response)
    except (IndexError, json.JSONDecodeError):
        print(f"Error parsing response: {response}")
        return None


def response_to_dict(response):
    data_dict = {
        'Vertragsnummer': "", 
        'Adresse Verbrauchsstelle': "", 
        'Verbrauchte Menge': "", 
        'Start abgerechneter Zeitraum': "", 
        'Ende abgerechneter Zeitraum': "", 
        'Energieart': "", 
        'Fehler': ""
    }

    temp_dict = extract_dict_from_response(response)
    if temp_dict is None:
        data_dict['Fehler'] = "True"
        return data_dict
    print(temp_dict)
    data_dict['Vertragsnummer'] = temp_dict['Vertragsnummer']
    data_dict['Adresse Verbrauchsstelle'] = ['Adresse Verbrauchsstelle']
    data_dict['Verbrauchte Menge'] = temp_dict['Verbrauchte Menge']
    data_dict['Start abgerechneter Zeitraum'] = temp_dict['Start abgerechneter Zeitraum']
    data_dict['Ende abgerechneter Zeitraum'] = temp_dict['Ende abgerechneter Zeitraum']
    data_dict['Energieart'] = temp_dict['Energieart']
    data_dict['Fehler'] = temp_dict['Fehler']
    return data_dict


In [ ]:
def extract_data_from_text(text, max_text_size=1500):
    # Ensure text does not exceed the maximum size allowed by the API
    text = text[:max_text_size]
    prompt = global_prompt
    headers = {
        'Authorization': f'Bearer lm-studio',
        'Content-Type': 'application/json'
    }

    data = {
        "model": "QuantFactory/Meta-Llama-3-8B-Instruct-GGUF-v2",
        "messages": [
            {"role": "system", "content": prompt},
            {"role": "user", "content": text}
        ],
        "temperature": 0
    }

    response = requests.post("http://localhost:1234/v1/chat/completions", headers=headers, json=data)
    
    # Print the full response for debugging
    print("Full response from LMStudio API:")
    print(response.text)
    
    if response.status_code != 200:
        print(f"Error: Received status code {response.status_code}")
        return None

    response_json = response.json()
    
    if 'choices' not in response_json or not response_json['choices']:
        print("Error: 'choices' key not found in the response")
        return None

    extracted_data = response_json['choices'][0]['message']['content'].strip()
    return extracted_data  # Remove any extra newlines at the end

In [ ]:
global_prompt = '''Du bekommst gleich einen String, der mit OCR aus einer Rechnung extrahiert wurde. Es handelt sich um eine einzelne Rechnung, die auch aus mehreren Seiten bestehen kann. Extrahiere die folgenden Daten aus diesem Text und gib sie im JSON-Format für die gesamte Rechnung aus. Gebe nur ein einzelnes JSON-Objekt aus. Hier ist eine Erklärung dazu, wie das JSON Objekt aussehen soll und was in die Felder rein soll: {
      "Vertragsnummer": "Hier soll die Vertragsnummer stehen. Diese ist in der Regel ein Integer, könnte aber in Ausnahmefällen auch andere Zeichen enthalten",
      "Adresse Verbrauchsstelle": "Das hier ist die Adresse der Verbrauchsstelle. Verwechsele sie nicht mit der Rechnungsadresse! Sie besteht aus Straße, Hausnummer, Postleitzahl und Ort",
      "Verbrauchte Menge": "Hier soll die verbrauchte Menge in Kilowattstunden (kWh) stehen. Wenn die Menge in Megawattstungen (MWh) angegeben ist, multipliziere mal 1000",
      "Start abgerechneter Zeitraum": "Gebe hier den Anfang des Abgerechneten Zeitraums als JJJJ-MM-TT an",
      "Ende abgerechneter Zeitraum": "Gebe hier das Ende des Abgerechneten Zeitraums als JJJJ-MM-TT an",
      "Energieart": "Hier soll entweder Gas oder Strom stehen",
      "Fehler": "Gib hier an, ob du bestimmte Daten nicht gefunden hast. Gib dann True an, ansonsten False. Damit wollen wir fehlerhafte Daten filtern"
      }
      Bitte gebe nur den JSON-String aus. Achte darauf, dass nur ein JSON-Objekt ausgegeben wird. Wenn ein Wert nicht gefunden wird, dann gib "nicht gefunden" an. Wenn Daten doppelt gefunden werden, dann fasse die Ergebnisse zusammen und gib nur den besten Treffer aus.'''



In [ ]:


if __name__ == "__main__":
    pdf_folder = '../data'  # Folder where the PDF files are located

    extracted_data = dict()

    pdf_files = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder) if f.endswith('.pdf')]
    for file_path in pdf_files:
        if not os.path.exists(pdf_folder):
            print(f"The folder '{pdf_folder}' does not exist.")
        else:
            # Process PDFs and extract data
            extracted_data[file_path] = response_to_dict(process_pdf(file_path))

    print(extracted_data)


In [ ]:

    output_df = pd.DataFrame.from_dict(extracted_data, orient='index')
    output_df    


In [ ]:
    # output_file_path = 'data/extracted_data.xlsx'
    # output_df.to_excel(output_file_path, index=False)
    # print(f'Excel file created: {output_file_path}')